In [2]:
import pandas as pd
import re
import os

In [20]:
def parsing_vina(vinaFF) -> pd.DataFrame:
    with open(vinaFF) as f:
        vina_output = [line.strip() for line in f.readlines()]
        
        start_index = None
        ligand = receptor = ""
        for i, line in enumerate(vina_output):
            if line.startswith("mode"):
                start_index = i
                break
            elif line.startswith("Ligand") and re.search(r"\bLigand\b", line):
                ligand = line.split()[1].split(".")[0]
            elif line.startswith("Ligand") and re.search(r"\bLigands\b", line):
                ligands = vina_output[i+1:i+3]
                ligand = [ligand.split()[-1].split(".")[0] for ligand in ligands]
                ligand = "_".join(ligand)
            elif line.startswith("Rigid receptor"):
                receptor = line.split()[2].split("-")[2]
           
        docking_data = []
        if start_index is not None:
            aff_binding = vina_output[start_index + 3:]
            for record in aff_binding:
                parts = record.split()
                mode = parts[0]
                affinity = parts[1]
                rmsd_lb = parts[2]
                rmsd_ub = parts[3]
                if ligand and receptor is not None:
                    docking_data.append({'mode': mode, "receptor": receptor, "ligand": ligand, 'affinity': affinity, 'rmsd_lb': rmsd_lb, 'rmsd_ub': rmsd_ub})
        docking_df = pd.DataFrame(docking_data)
        # print(docking_df.to_string(index = False))
        return docking_df

In [61]:
# Receptor: WT + PRPP + PRPP_5FU
receptor_ligand_docking = []
for vina_output in os.listdir("./input/"):
    vina_output = os.path.join("input", vina_output)
    docking_output = parsing_vina(vina_output)
    receptor_ligand_docking.append(docking_output.loc[:, ["receptor", "ligand", "affinity"]])
# Concatenate all df in the list row-wise
merged_df = pd.concat(receptor_ligand_docking, ignore_index = True)
merged_df = merged_df.sort_values(by = ["receptor", "ligand"])
merged_df.reset_index(drop = True, inplace = True)


In [68]:
merged_df.replace(
    {
        "192R4": "R4",
        "R214T": "R1"
    }, inplace = True)

In [69]:
merged_df

,receptor,ligand,affinity
0,R4,5FU,-4.156
1,R4,5FU,-4.155
2,R4,5FU,-4.028
3,R4,5FU,-3.98
4,R4,5FU,-3.945
...,...,...,...
175,WT,PRPP_5FU,-8.976
176,WT,PRPP_5FU,-8.921
177,WT,PRPP_5FU,-8.916
178,WT,PRPP_5FU,-8.899


In [70]:
merged_df.to_csv("test.csv", index = False)